In [ ]:
import requests
import pandas as pd
import json
import datetime, time
import csv
import re
import numpy as np
import math
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, roc_curve, hamming_loss, cohen_kappa_score
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.2MB/s 


In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Reddit'

/content/drive/My Drive/Reddit


## Sentiment Analysis - DON'T RUN

### Vader

In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.4MB/s 


In [ ]:
import vaderSentiment

In [ ]:
pip install --user -U nltk

     |████████████████████████████████| 1.5MB 3.4MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449907 sha256=483918d3c5ece1bc176144dd588cfeafe5d58e5632af21831eace43bb7d1a5d5
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(strip_handles=True,reduce_len=True)

In [ ]:
def remove_stopwords(word_tokens):
  stop_words = set(stopwords.words('english')) 
    
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
  filtered_sentence = [] 
    
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  return(filtered_sentence)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
def get_vader_score(df, col, score_type):
  vader_score = df[col].apply(vader_analyzer.polarity_scores)

  if score_type == 'compound':
     return([d['compound'] for d in vader_score])
  elif score_type == 'neg': 
      return([d['neg'] for d in vader_score])
  elif score_type == 'neu': 
      return([d['neu'] for d in vader_score])
  else:
      return([d['pos'] for d in vader_score])

### Utils for Data Cleansing

In [ ]:
!pip install contractions
import contractions

     |████████████████████████████████| 245kB 5.0MB/s 
     |████████████████████████████████| 317kB 49.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81702 sha256=3734937643b600479e3ae49188b6d221b43e071390f9c71372a15ecbe74590ca
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_URL(text):
    """Remove URLs from a sample string"""
    return re.sub(r'http\S+ | www\S+', "", text)
def remove_special_chars(text):
    text = re.sub("\s+"," ",text)
    text = re.sub("[^A-Za-z0-9.,;:']+",' ',text)
    return(text)

def clean_string(text):
  text = remove_URL(text)
  text = replace_contractions(text)
  text = remove_special_chars(text)
  return(text)

### TextBlob

In [ ]:
!pip install -U textblob

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)


In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
def get_textblob_score(df, col):
  from textblob import TextBlob
  from textblob.sentiments import NaiveBayesAnalyzer
  textblob_score = df[col].apply(TextBlob,analyzer=NaiveBayesAnalyzer())
  return([d.sentiment.p_pos for d in textblob_score])

In [ ]:
def clean_df(df,col,phrase='[deleted]'):
  df.dropna(inplace=True)
  df = df[df[col] != phrase]
  df.loc[:,col] = df[col].apply(clean_string)
  df.dropna(inplace=True)
  df.reset_index(inplace=True)
  df.drop(columns='index',inplace=True)
  return(df)

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(1,102):
  df = pd.read_csv('reddit_data'+str(i)+'.csv', dtype={'created_utc':'str','subreddit':'str','body':'str'})
  df = clean_df(df,'body','[deleted]')
  df['vader_compound_score'] = get_vader_score(df, 'body', 'compound')
  df['textblob_pos_score'] = get_textblob_score(df, 'body')
  df.to_csv('reddit_sentiment'+str(i)+'.csv')
  print('Done for file',i)

Done for file 1
Done for file 2
Done for file 3
Done for file 4
Done for file 5
Done for file 6
Done for file 7
Done for file 8
Done for file 9
Done for file 10
Done for file 11
Done for file 12
Done for file 13
Done for file 14
Done for file 15
Done for file 16
Done for file 17
Done for file 18
Done for file 19
Done for file 20
Done for file 21
Done for file 22
Done for file 23
Done for file 24
Done for file 25
Done for file 26
Done for file 27
Done for file 28
Done for file 29
Done for file 30
Done for file 31
Done for file 32
Done for file 33
Done for file 34


## Further processing - DON'T RUN

In [ ]:
reddit_daily_sentiment = pd.read_csv('reddit_sentiment1.csv',dtype={'Unnamed: 0':'int','created_utc':'int','subreddit':'str','body':'str','vader_compound_score':'float','textblob_pos_score':'float'})
reddit_daily_sentiment.drop(columns='Unnamed: 0',inplace=True)
reddit_daily_sentiment['date'] = reddit_daily_sentiment['created_utc'].apply(lambda ts: datetime.datetime.utcfromtimestamp(ts))
reddit_daily_sentiment.set_index('date',inplace=True)
reddit_daily_sentiment = reddit_daily_sentiment.groupby(pd.Grouper(freq='D')).mean()
for i in range(2,102):
  df = pd.read_csv('reddit_sentiment'+str(i)+'.csv',dtype={'Unnamed: 0':'int','created_utc':'int','subreddit':'str','body':'str','vader_compound_score':'float','textblob_pos_score':'float'})
  df.drop(columns='Unnamed: 0',inplace=True)
  df['date'] = df['created_utc'].apply(lambda ts: datetime.datetime.utcfromtimestamp(ts))
  df.set_index('date',inplace=True)
  df_res = df.groupby(pd.Grouper(freq='D')).mean()
  #df_res.to_csv('reddit_daily_sentiment'+str(i)+'.csv')
  reddit_daily_sentiment = pd.concat([reddit_daily_sentiment,df_res])
  print('Done for file',i)

Done for file 2
Done for file 3
Done for file 4
Done for file 5
Done for file 6
Done for file 7
Done for file 8
Done for file 9
Done for file 10
Done for file 11
Done for file 12
Done for file 13
Done for file 14
Done for file 15
Done for file 16
Done for file 17
Done for file 18
Done for file 19
Done for file 20
Done for file 21
Done for file 22
Done for file 23
Done for file 24
Done for file 25
Done for file 26
Done for file 27
Done for file 28
Done for file 29
Done for file 30
Done for file 31
Done for file 32
Done for file 33
Done for file 34
Done for file 35
Done for file 36
Done for file 37
Done for file 38
Done for file 39
Done for file 40
Done for file 41
Done for file 42
Done for file 43
Done for file 44
Done for file 45
Done for file 46
Done for file 47
Done for file 48
Done for file 49
Done for file 50
Done for file 51
Done for file 52
Done for file 53
Done for file 54
Done for file 55
Done for file 56
Done for file 57
Done for file 58
Done for file 59
Done for file 60
Done 

In [ ]:
reddit_daily_sentiment.dropna(inplace=True)

In [ ]:
reddit_daily_sentiment.to_csv('reddit_daily_sentiment.csv')

Merging all files

In [ ]:
def merge_files(freq):
  fout=open("reddit_"+freq+"_sentiment.csv","a")
  # first file:
  for line in open("reddit_"+freq+"_sentiment1.csv"):
      fout.write(line)
  # now the rest:    
  for num in range(2,102):
      f = open("reddit_"+freq+"_sentiment"+str(num)+".csv")
      f.__next__() # skip the header, new in python 3
      for line in f:
          fout.write(line)
      f.close() # not really needed
  fout.close()

## Final Analysis

### Monthly predictions

In [ ]:
df_sentiment = pd.read_csv('reddit_monthly_sentiment.csv')
df_stock_indices = pd.read_csv('stock_indices_2009-2019.csv',index_col='date',parse_dates=True)

In [ ]:
df_sentiment = df_sentiment.groupby('date').mean()

In [ ]:
df_stock_indices_monthly = df_stock_indices.groupby(pd.Grouper(freq='M')).mean()

In [ ]:
df_stock_indices_monthy_returns = (df_stock_indices_monthly.shift(1) - df_stock_indices_monthly)/ df_stock_indices_monthly

In [ ]:
df_stock_indices_monthy_returns.dropna(inplace=True)

In [ ]:
df_monthly_sentiment = df_sentiment.shift(-1)

In [ ]:
df_monthly_sentiment.dropna(inplace=True)

In [ ]:
df_monthly_sentiment['predicted_up'] = df_monthly_sentiment.vader_compound_score > 0.05

In [ ]:
df_monthly_sentiment

,created_utc,vader_compound_score,textblob_pos_score,predicted_up
date,,,,
2009-01-31,1.234614e+09,0.150483,0.730683,True
2009-02-28,1.237146e+09,0.167607,0.647738,True
2009-03-31,1.239931e+09,0.046171,0.636454,False
2009-04-30,1.242986e+09,0.027224,0.649321,False
2009-05-31,1.245485e+09,0.064500,0.726431,True
...,...,...,...,...
2019-06-30,1.563292e+09,0.076853,0.540346,True
2019-07-31,1.565868e+09,0.053090,0.534864,True
2019-08-31,1.568626e+09,0.054687,0.538876,True


In [ ]:
df_stock_indices_monthly_returns = (df_stock_indices_monthly.shift(1) - df_stock_indices_monthly) / df_stock_indices_monthly

In [ ]:
df_stock_indices_monthly_returns.dropna(inplace=True)

In [ ]:
df_stock_indices_monthly_returns['SPX_up'] = df_stock_indices_monthly_returns['SPX'] > 0

In [ ]:
df_stock_indices_monthly_returns['DJIA_up'] = df_stock_indices_monthly_returns['DJIA'] > 0

In [ ]:
df_stock_indices_monthly_returns['Nasdaq_up'] = df_stock_indices_monthly_returns['Nasdaq'] > 0

Making Predictions

In [ ]:
n_predictions = df_stock_indices_monthly_returns.shape[0]

In [ ]:
(n_predictions - (df_monthly_sentiment['predicted_up'].values == df_stock_indices_monthly_returns['DJIA_up'].values).sum())
 / n_predictions

0.6692307692307692

In [ ]:
print(classification_report(df_stock_indices_monthly_returns['DJIA_up'].values,df_monthly_sentiment['predicted_up'].values))

              precision    recall  f1-score   support

       False       1.00      0.02      0.04        89
        True       0.32      1.00      0.49        41

    accuracy                           0.33       130
   macro avg       0.66      0.51      0.26       130
weighted avg       0.79      0.33      0.18       130



In [ ]:
(n_predictions - (df_monthly_sentiment['predicted_up'].values == df_stock_indices_monthly_returns['SPX_up'].values).sum())
 / n_predictions

0.6692307692307692

In [ ]:
print(classification_report(df_stock_indices_monthly_returns['SPX_up'].values,df_monthly_sentiment['predicted_up'].values))

              precision    recall  f1-score   support

       False       1.00      0.02      0.04        89
        True       0.32      1.00      0.49        41

    accuracy                           0.33       130
   macro avg       0.66      0.51      0.26       130
weighted avg       0.79      0.33      0.18       130



In [ ]:
(n_predictions - (df_monthly_sentiment['predicted_up'].values == df_stock_indices_monthly_returns['Nasdaq_up'].values).sum()) / n_predictions

0.7230769230769231

In [ ]:
print(classification_report(df_stock_indices_monthly_returns['Nasdaq_up'].values,df_monthly_sentiment['predicted_up'].values))

              precision    recall  f1-score   support

       False       1.00      0.02      0.04        96
        True       0.27      1.00      0.42        34

    accuracy                           0.28       130
   macro avg       0.63      0.51      0.23       130
weighted avg       0.81      0.28      0.14       130



### Daily predictions

In [ ]:
df_stock_indices = pd.read_csv('stock_indices_2009-2019.csv',index_col='date',parse_dates=True)
df_stock_indices_daily = df_stock_indices.groupby(pd.Grouper(freq='D')).mean()
df_stock_indices_daily.dropna(inplace=True)

In [ ]:
df_stock_indices_daily_returns = (df_stock_indices_daily.shift(1) - df_stock_indices_daily)/df_stock_indices_daily

In [ ]:
df_stock_indices_daily_returns.dropna(inplace=True)

In [ ]:
def _get_prior_sentiment(date,df_sentiment,score_col, n_days):
  """
    date form: 'YYYY-MM-DD' AND the 'date' column must be the index
    score_col: the column with the score to get
    n_days: number days to look back
  """
  return(df_sentiment[df_sentiment.index < date][score_col][-n_days:].mean())

In [ ]:
reddit_daily_sentiment = pd.read_csv('reddit_daily_sentiment.csv',index_col='date',parse_dates=True)

In [ ]:
daily_predictions = [_get_prior_sentiment(date,reddit_daily_sentiment,'vader_compound_score',14) for date in df_stock_indices_daily_returns.index]

In [ ]:
daily_predictions = np.array(daily_predictions) > 0.05

In [ ]:
df_stock_indices_daily_returns['SPX_up'] = df_stock_indices_daily_returns['SPX'] > 0
df_stock_indices_daily_returns['DJIA_up'] = df_stock_indices_daily_returns['DJIA'] > 0
df_stock_indices_daily_returns['Nasdaq_up'] = df_stock_indices_daily_returns['Nasdaq'] > 0

In [ ]:
n_predictions = len(daily_predictions)

In [ ]:
(n_predictions - (daily_predictions == df_stock_indices_daily_returns['SPX_up'].values).sum()) / n_predictions

0.549908592321755

In [ ]:
print(classification_report(df_stock_indices_daily_returns['SPX_up'].values,daily_predictions))

              precision    recall  f1-score   support

       False       0.40      0.01      0.01      1499
        True       0.45      0.99      0.62      1236

    accuracy                           0.45      2735
   macro avg       0.43      0.50      0.32      2735
weighted avg       0.42      0.45      0.29      2735



In [ ]:
(n_predictions - (daily_predictions == df_stock_indices_daily_returns['DJIA_up'].values).sum()) / n_predictions

0.5451553930530164

In [ ]:
print(classification_report(df_stock_indices_daily_returns['DJIA_up'].values,daily_predictions))

              precision    recall  f1-score   support

       False       0.44      0.01      0.01      1488
        True       0.45      0.99      0.62      1247

    accuracy                           0.45      2735
   macro avg       0.45      0.50      0.32      2735
weighted avg       0.45      0.45      0.29      2735



In [ ]:
(n_predictions - (daily_predictions == df_stock_indices_daily_returns['Nasdaq_up'].values).sum()) / n_predictions

0.5608775137111517

In [ ]:
print(classification_report(df_stock_indices_daily_returns['Nasdaq_up'].values,daily_predictions))

              precision    recall  f1-score   support

       False       0.40      0.01      0.01      1529
        True       0.44      0.99      0.61      1206

    accuracy                           0.44      2735
   macro avg       0.42      0.50      0.31      2735
weighted avg       0.42      0.44      0.28      2735

